# Model Selection 모듈

- Train/Test 데이터를 분리하지 않고 머신러닝 수행

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:
iris = load_iris()
dt_clf = DecisionTreeClassifier()
dt_clf.fit(iris.data, iris.target)

DecisionTreeClassifier()

In [3]:
pred = dt_clf.predict(iris.data)
accuracy_score(iris.target, pred)

1.0

- cross_validate() method

In [4]:
from sklearn.model_selection import cross_validate

In [5]:
dtc = DecisionTreeClassifier()
cross_validate(dtc, iris.data, iris.target)

{'fit_time': array([0.00103211, 0.        , 0.        , 0.        , 0.00097036]),
 'score_time': array([0.        , 0.00096869, 0.00102639, 0.00099492, 0.        ]),
 'test_score': array([0.96666667, 0.96666667, 0.9       , 0.96666667, 1.        ])}

In [6]:
dtc = DecisionTreeClassifier()
cross_validate(dtc, iris.data, iris.target, return_train_score=True)

{'fit_time': array([0.00099778, 0.        , 0.00102401, 0.0009706 , 0.00102401]),
 'score_time': array([0.00099659, 0.00099635, 0.        , 0.        , 0.        ]),
 'test_score': array([0.96666667, 0.96666667, 0.9       , 1.        , 1.        ]),
 'train_score': array([1., 1., 1., 1., 1.])}

- Train/Test 데이터 셋을 분리

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, test_size=0.2, random_state=2021
)

In [9]:
import pandas as pd
pd.Series(y_test).value_counts()

0    14
1    10
2     6
dtype: int64

- Stratified 분리

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, test_size=0.2, random_state=2021,
    stratify=iris.target
)
pd.Series(y_test).value_counts()

0    10
1    10
2    10
dtype: int64

- cross_val_score() method

In [11]:
from sklearn.model_selection import cross_val_score

In [12]:
# 성능 지표는 정확도(accuracy), 교차검증 세트는 5개
cross_val_score(dtc, iris.data, iris.target, cv=5)

array([0.96666667, 0.96666667, 0.9       , 1.        , 1.        ])

In [13]:
import numpy as np 
scores = cross_val_score(dtc, iris.data, iris.target, cv=5)
np.mean(scores)

0.9600000000000002

### GridSearchCV
- 교차 검증과 최적 하이퍼 파라메터 튜닝을 한꺼번에 수행

In [14]:
dtc = DecisionTreeClassifier(random_state=2021)

In [15]:
# parameter 를 dictionary 형태로 설정
params = {
    'max_depth': [2, 3, 4, 5],
    'min_samples_split': [2, 3]
}

In [16]:
from sklearn.model_selection import GridSearchCV
grid_dtc = GridSearchCV(dtc, param_grid=params, cv=3)

In [17]:
grid_dtc.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=2021),
             param_grid={'max_depth': [2, 3, 4, 5],
                         'min_samples_split': [2, 3]})

In [18]:
grid_dtc.cv_results_

{'mean_fit_time': array([0.00165852, 0.00066352, 0.00065486, 0.00099619, 0.00065565,
        0.00099723, 0.0006783 , 0.        ]),
 'std_fit_time': array([4.68408976e-04, 4.69183170e-04, 4.63186303e-04, 2.52861110e-05,
        4.63752799e-04, 2.54064677e-06, 4.79893466e-04, 0.00000000e+00]),
 'mean_score_time': array([0.00033514, 0.00066837, 0.00033092, 0.        , 0.00033212,
        0.        , 0.        , 0.00067639]),
 'std_score_time': array([0.00047396, 0.00047344, 0.000468  , 0.        , 0.00046968,
        0.        , 0.        , 0.00047847]),
 'param_max_depth': masked_array(data=[2, 2, 3, 3, 4, 4, 5, 5],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_min_samples_split': masked_array(data=[2, 3, 2, 3, 2, 3, 2, 3],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 2, 'min_samples_split'

In [19]:
# 최적 파라미터
grid_dtc.best_params_

{'max_depth': 2, 'min_samples_split': 2}

In [20]:
# 최고 정확도
grid_dtc.best_score_

0.9583333333333334

In [21]:
# 최고 정확도를 가지는 최적 파라미터로 학습한 estimator
best_estimator = grid_dtc.best_estimator_
best_estimator.score(X_test, y_test)

0.9